# Training a model
see README.md for details

In [2]:
from tqdm import tqdm
import glob
from transformers import T5ForConditionalGeneration, AutoTokenizer #, TrainingArguments, Trainer
import torch

from config import data_root, model_root

In [3]:
torch.cuda.is_available()

True

In [4]:
# training_args = TrainingArguments(per_device_train_batch_size=1, gradient_accumulation_steps=4, **default_args)

NameError: name 'TrainingArguments' is not defined

In [5]:
mname = 'google/byt5-small'
model = T5ForConditionalGeneration.from_pretrained(mname)
tokenizer = AutoTokenizer.from_pretrained(mname)
type(model)

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

transformers.models.t5.modeling_t5.T5ForConditionalGeneration

In [6]:
input_list = []
label_list = []

for fname in tqdm(glob.glob(f"{data_root}/00-Prolog.train")):
# for fname in tqdm(glob.glob(f"{data_root}/*.train")):
    print(fname)
    with open(fname) as fin:
        for i, row in enumerate(fin):
            line = row[:-1].split("\t")
            input_list += [line[0]]
            label_list += [line[1]]
len(input_list)

100%|██████████| 1/1 [00:00<00:00, 226.83it/s]

../stanza_pos/00-Prolog.train


392

In [7]:

model_inputs = tokenizer(input_list, padding="longest", return_tensors="pt")

labels_dict = tokenizer(label_list, padding="longest", return_tensors="pt")
labels = labels_dict.input_ids

m = model(**model_inputs, labels=labels)
# m = model(input_list, labels=label_list)

In [9]:
model.save_pretrained(f"{model_root}/byT5-ocs-stanzapos-Prolog")